In [1]:
import pandas as pd 
df=pd.read_csv('/kaggle/input/kge-text-51-100/kge_text_51-100.csv')
df

,head,type,tail
0,chips,part of,sandwich
1,sandwich,has part,chips
2,chip,part of,sandwich
3,salariess,subclass of,wage
4,wage,different from,salariess
...,...,...,...
140,toilet,part of,bathroom
141,shower,part of,bathroom
142,hat,has part,white
143,green,has part,white


In [2]:
!pip install graphviz

In [3]:
import pandas as pd
from graphviz import Digraph

# Create a Digraph object
dot = Digraph()

# Add nodes and edges to the Digraph
for index, row in df.iterrows():
    dot.node(row['head'])
    dot.node(row['tail'])
    dot.edge(row['head'], row['tail'], label=row['type'])

# Render and display the graph in the notebook
dot.render('knowledge_graph', format='png', view=True)


'knowledge_graph.png'

In [4]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

# Assuming df is already defined
# df = pd.DataFrame({'head': ..., 'tail': ...})

# Create a directed graph
G = nx.from_pandas_edgelist(df, source='head', target='tail', edge_attr=True, create_using=nx.DiGraph())

# Adjust spring layout to spread out the nodes more
pos = nx.spring_layout(G, k=0.3, iterations=50)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
    ),
    text=list(G.nodes()))

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Knowledge Graph',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20, l=5, r=5, t=40),
                annotations=[ dict(
                    text="Knowledge Graph using Plotly",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False),
                yaxis=dict(showgrid=False, zeroline=False),
                width=1000,  # Increased width
                height=1000  # Increased height
                )
             )

fig.show()


Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'knowledge_graph.png'


In [5]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 3.9 MB/s eta 0:00:0000:0100:01


In [ ]:
from neo4j import GraphDatabase
import pandas as pd

# Connect to Neo4j
uri = "neo4j+s://6a0504cc.databases.neo4j.io"
driver = GraphDatabase.driver(uri, auth=("neo4j", "aq_czfyU7NpukHH2sQM6MyYOWvfxgiRYuvjLAPiTTtU"))

def create_graph(tx, head, relation, tail):
    tx.run("MERGE (a:Node {name: $head}) "
           "MERGE (b:Node {name: $tail}) "
           "MERGE (a)-[r:RELATION {type: $relation}]->(b)",
           head=head, tail=tail, relation=relation)

# Write the data to Neo4j
with driver.session() as session:
    for index, row in df.iterrows():
        session.write_transaction(create_graph, row['head'], row['type'], row['tail'])

# Close the connection
driver.close()

# To visualize, you need to use Neo4j browser at http://localhost:7474/


/tmp/ipykernel_33/1506400838.py:17: DeprecationWarning:

write_transaction has been renamed to execute_write

